In [16]:
import torch
import torch.nn.functional as F

In [3]:
B = 3
L = 8
Z = 5

In [4]:
latent_buffer = torch.randn(B,L,Z)
i = 1
latent_buffer.shape, latent_buffer[:, i : i + 1].shape

(torch.Size([3, 8, 5]), torch.Size([3, 1, 5]))

In [ ]:
skill = torch.randn(B, L, Z, Z)
probs = F.softmax(skill, dim = -1)
# probs.shape #torch.Size([3, 8, 5, 5])
        # uniform = tf.ones_like(probs) / probs.shape[-1]
        # probs = (1 - self._unimix) * probs + self._unimix * uniform
dist = OneHotDist(probs=probs)

torch.Size([3, 8, 5, 5])

In [12]:
import torch
import torch.nn.functional as F
from torch.distributions import Categorical


class OneHotDist(torch.nn.Module):
    def __init__(self, logits: torch.Tensor):
        """
        Args:
            logits: Tensor of shape [B, K, K]
        """
        super().__init__()
        self.logits = logits  # [B, K, K]
        assert logits.dim() == 3, "Expected logits of shape [B, M, K]"

    def sample(self):
        """
        Returns:
            A differentiable one-hot sample of shape [B, M, K]
        """
        B, M, K = self.logits.shape

        # Flatten for sampling: [B*M, K]
        flat_logits = self.logits.reshape(-1, K)

        # Sample indices: [B*M]
        indices = Categorical(logits=flat_logits).sample()

        # One-hot encode: [B*M, K]
        hard_sample = F.one_hot(indices, num_classes=K).float()

        # Reshape back: [B, M, K]
        hard_sample = hard_sample.reshape(B, M, K)

        # Straight-through estimator:
        # Forward: hard one-hot
        # Backward: gradient through softmax
        probs = F.softmax(self.logits, dim=-1)
        return (hard_sample - probs).detach() + probs  # ST-trick

    def log_prob(self, one_hot_action: torch.Tensor):
        """
        Args:
            one_hot_action: Tensor of shape [B, M, K], one-hot encoded actions

        Returns:
            log_prob: Tensor of shape [B, M], log probability per categorical
        """
        log_probs = F.log_softmax(self.logits, dim=-1)
        return (log_probs * one_hot_action).sum(dim=-1)  # [B, M]


In [14]:
logits = torch.randn(2, 8, 8)

dist = OneHotDist(logits)
skill = dist.sample()  # [2, 8, 8] — each is one-hot, differentiable

# Compute log-prob if needed
logp = dist.log_prob(skill)  # [2, 8]
skill[0], logp[0]

(tensor([[0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0.]]),
 tensor([-2.6144, -1.7148, -1.1793, -0.7010, -1.8167, -1.2877, -2.1071, -1.4316]))

In [ ]:
logits = torch.randn(2, 8, 8)
logits.shapoe

AttributeError: 'Tensor' object has no attribute 'shapoe'